In [1]:
import sqlite3
import numpy as np
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

db = sqlite3.connect('L22DB_ex.sqlite')
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS candidates")
cursor.execute("DROP TABLE IF EXISTS contributors")
cursor.execute("PRAGMA foreign_keys=1")

cursor.execute('''CREATE TABLE candidates (
               id INTEGER PRIMARY KEY NOT NULL, 
               first_name TEXT, 
               last_name TEXT, 
               middle_init TEXT, 
               party TEXT NOT NULL)''')

db.commit() # Commit changes to the database

cursor.execute('''CREATE TABLE contributors (
          id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
          last_name TEXT, 
          first_name TEXT, 
          middle_name TEXT, 
          street_1 TEXT, 
          street_2 TEXT, 
          city TEXT, 
          state TEXT, 
          zip TEXT, 
          amount REAL, 
          date DATETIME, 
          candidate_id INTEGER NOT NULL, 
          FOREIGN KEY(candidate_id) REFERENCES candidates(id))''')

db.commit()

with open ("candidates.txt") as candidates:
    next(candidates) # jump over the header
    for line in candidates.readlines():
        cid, first_name, last_name, middle_name, party = line.strip().split('|')
        vals_to_insert = (int(cid), first_name, last_name, middle_name, party)
        cursor.execute('''INSERT INTO candidates 
                  (id, first_name, last_name, middle_init, party)
                  VALUES (?, ?, ?, ?, ?)''', vals_to_insert)

with open ("contributors.txt") as contributors:
    next(contributors)
    for line in contributors.readlines():
        cid, last_name, first_name, middle_name, street_1, street_2, \
            city, state, zip_code, amount, date, candidate_id = line.strip().split('|')
        vals_to_insert = (last_name, first_name, middle_name, street_1, street_2, 
                          city, state, int(zip_code), amount, date, candidate_id)
        cursor.execute('''INSERT INTO contributors (last_name, first_name, middle_name, 
                           street_1, street_2, city, state, zip, amount, date, candidate_id) 
                           VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', vals_to_insert)

candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(candidates)")]
contributor_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")]

def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = dict()
    for i, col_name in enumerate(cols):
        framelist[col_name] = [col[i] for col in q]
    return pd.DataFrame.from_dict(framelist)

from IPython.display import display

In [2]:
# Sample query
query = '''SELECT * FROM contributors WHERE candidate_id = (SELECT id from candidates WHERE last_name = "Obama")'''
viz_tables(contributor_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,26,Buckler,Steve,,24351 Armada Dr,,Dana Point,CA,926291306,50.00,2007-07-30,20
1,27,Buckler,Steve,,24351 Armada Dr,,Dana Point,CA,926291306,25.00,2007-08-16,20
2,28,Buckheit,Bruce,,8904 KAREN DR,,FAIRFAX,VA,220312731,100.00,2007-09-19,20
3,29,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,2300.00,2007-08-14,20
4,30,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,-2300.00,2007-08-14,20
5,31,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,4600.00,2007-08-14,20
6,32,Buck,Thomas,,4206 Terrace Street,,Kansas City,MO,64111,100.00,2007-09-25,20
7,33,Buck,Jay,K.,1855 Old Willow Rd Unit 322,,Northfield,IL,600932918,200.00,2007-09-12,20
8,34,Buck,Blaine,M,45 Eaton Ave,,Camden,ME,48431752,2300.00,2007-09-30,20
9,35,Buck,Barbara,,1780 NE 138th St,,North Miami,FL,331811316,50.00,2007-09-13,20


In [4]:
# Inner Join
cols = ["contributors.last_name", "contributors.first_name", "contributors.amount", "candidates.last_name"]
query = '''SELECT contributors.last_name, contributors.first_name, amount, candidates.last_name 
           FROM contributors 
           INNER JOIN candidates 
           ON candidates.id = contributors.candidate_id'''
display(viz_tables(cols, query))

# Inner Join and select last name
cols = ["contributors.last_name", "contributors.first_name", "contributors.amount", "candidates.last_name"]
query = '''SELECT contributors.last_name, contributors.first_name, amount, candidates.last_name 
           FROM contributors 
           INNER JOIN candidates 
           ON candidates.id = contributors.candidate_id 
           WHERE candidates.last_name = "Obama"'''
display(viz_tables(cols, query))

,contributors.last_name,contributors.first_name,contributors.amount,candidates.last_name
0,Agee,Steven,500.0,Huckabee
1,Ahrens,Don,250.0,Huckabee
2,Ahrens,Don,50.0,Huckabee
3,Ahrens,Don,100.0,Huckabee
4,Akin,Charles,100.0,Huckabee
...,...,...,...,...
170,ABESHAUS,MERRILL,120.0,McCain
171,ABRAHAM,GEORGE,800.0,McCain
172,ABRAHAMSON,PETER,50.0,McCain
173,ABRAHAM,SALEM,1000.0,McCain


,contributors.last_name,contributors.first_name,contributors.amount,candidates.last_name
0,Buckler,Steve,50.00,Obama
1,Buckler,Steve,25.00,Obama
2,Buckheit,Bruce,100.00,Obama
3,Buckel,Linda,2300.00,Obama
4,Buckel,Linda,-2300.00,Obama
5,Buckel,Linda,4600.00,Obama
6,Buck,Thomas,100.00,Obama
7,Buck,Jay,200.00,Obama
8,Buck,Blaine,2300.00,Obama
9,Buck,Barbara,50.00,Obama


In [5]:
# Left Join Example

q = '''SELECT AVG(contributors.amount), candidates.last_name
       FROM contributors LEFT JOIN candidates
       ON contributors.candidate_id = candidates.id 
       WHERE contributors.amount > 0
       GROUP BY candidates.last_name'''
display(viz_tables(["Average Contribution", "candidates.last_name"], q))

,Average Contribution,candidates.last_name
0,170.217391,Clinton
1,772.000000,Huckabee
2,452.600000,McCain
3,695.949545,Obama
4,290.909091,Romney


## Class Exercises

### Left Join #1

| col1 | val1 | col2 | val2 |
|------|------|------|------|
| s1   | 23   | t1   | 60   |
| s2   | 7    | t3   | 22   |
| s3   | 15   | t3   | 22   |
| s4   | 31   | t1   | 60   |

### Left Join #2

| col1 | val1 | col2 | val2 |
|------|------|------|------|
| t1   | 60   | s1   | 23   |
| t1   | 60   | s4   | 31   |
| t2   | 14   | --   | --   |
| t3   | 22   | s2   | 7    |
| t3   | 22   | s3   | 7    |

## Additional Work with Pandas

In [7]:
# Using Pandas

# Read candidates
dfcand = pd.read_csv("candidates.txt", sep="|")
dfcand

# Read contributors
dfcontr = pd.read_csv("contributors.txt", sep="|")
dfcontr

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,NaN,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,NaN,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16
...,...,...,...,...,...,...,...,...,...,...,...,...
170,NaN,ABESHAUS,MERRILL,M.,1801 N. HEREFORD DRIVE,NaN,FLAGSTAFF,AZ,860011121,120.0,2008-01-16,37
171,NaN,ABRAHAM,GEORGE,NaN,P.O. BOX 1504,NaN,LAKE CHARLES,LA,706021504,800.0,2008-01-17,37
172,NaN,ABRAHAMSON,PETER,J.,1030 W. ROSCOE STREET,NaN,CHICAGO,IL,606572207,50.0,2008-01-25,37
173,NaN,ABRAHAM,SALEM,A.,P.O. BOX 7,NaN,CANADIAN,TX,790140007,1000.0,2008-01-17,37


In [9]:
# Quick deletion with pandas

del dfcontr['id']
dfcontr.head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16


In [13]:
# Creating a new database

dbp = sqlite3.connect('L22_pandas_DB.sqlite')
csr = dbp.cursor()
csr.execute("DROP TABLE IF EXISTS candidates")
csr.execute("DROP TABLE IF EXISTS contributors")
csr.execute("PRAGMA foreign_keys=1")

csr.execute('''CREATE TABLE candidates (
               id INTEGER PRIMARY KEY NOT NULL, 
               first_name TEXT, 
               last_name TEXT, 
               middle_name TEXT, 
               party TEXT NOT NULL)''')

dbp.commit() # Commit changes to the database

csr.execute('''CREATE TABLE contributors (
          id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
          last_name TEXT, 
          first_name TEXT, 
          middle_name TEXT, 
          street_1 TEXT, 
          street_2 TEXT, 
          city TEXT, 
          state TEXT, 
          zip TEXT, 
          amount REAL, 
          date DATETIME, 
          candidate_id INTEGER NOT NULL, 
          FOREIGN KEY(candidate_id) REFERENCES candidates(id))''')

dbp.commit()

In [14]:
dfcand.to_sql("candidates", dbp, if_exists="append", index=False)

In [15]:
dfcand.shape

(17, 5)

In [21]:
# Classic Query

query = '''SELECT * FROM candidates'''
csr.execute(query).fetchall()

# Query with Pandas

dfcand.query("first_name=='Mike' & party=='D'") # Option 1
dfcand[(dfcand.first_name=="Mike") & (dfcand.party=="D")] # Option 2
dfcand[dfcand.middle_name.notnull()] # Option 3
dfcand[dfcand.first_name.isin(['Mike', 'Hillary'])] # Option 4



,id,first_name,last_name,middle_name,party
2,34,Hillary,Clinton,R.,D
6,24,Mike,Gravel,NaN,D
7,16,Mike,Huckabee,NaN,R


In [22]:
# Pandas Exercises

# Populate contributors table
dfcontr.to_sql("contributors", dbp, if_exists="append", index=False)

# List where state is "VA" and amount is less than $400
display(dfcontr.query("state=='VA' & amount<400.00"))

# List entries with NULL
display(dfcontr[dfcontr.state.isnull()])

# List where state is either "TX" or "PA"
display(dfcontr[dfcontr.state.isin(['TX', 'PA'])])

# Find contributions between $10 and $50
display(dfcontr.query("10 < amount < 50"))

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
27,Buckheit,Bruce,NaN,8904 KAREN DR,NaN,FAIRFAX,VA,220312731,100.00,2007-09-19,20
77,Ranganath,Anoop,NaN,2507 Willard Drive,NaN,Charlottesville,VA,22903,-100.00,2008-04-21,32
88,Perreault,Louise,NaN,503 Brockridge Hunt Drive,NaN,Hampton,VA,23666,-34.08,2008-04-21,32
145,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,NaN,FAIRFAX,VA,220308440,50.00,2007-09-30,35


,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
125,BOURNE,TRAVIS,NaN,LAGE KAART 77,NaN,BRASSCHATT,NaN,2930,-500.0,2008-11-20,35


,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
65,BURGERT,RONALD,L.,5723 PLUMTREE DRIVE,NaN,DALLAS,TX,752524926,-1000.00,2008-03-05,22
70,BUCKLEY,WALTER,W.,1635 COUNTRY ROAD,NaN,BETHLEHEM,PA,180155718,-100.00,2008-03-05,22
71,BUCKLEY,MARJORIE,B.,1635 COUNTRY ROAD,NaN,BETHLEHEM,PA,180155718,-100.00,2008-03-05,22
79,Muse,Nina,Jo,2915 Toro Canyon Rd,NaN,Austin,TX,78746,-50.00,2008-04-21,32
93,Raught,Philip,M,4714 Plum Way,NaN,Pittsburgh,PA,15201,-1046.00,2008-04-21,32
94,Ferrara,Judith,D,1508 Waterford Road,NaN,Yardley,PA,19067,-1100.00,2008-04-21,32
95,Johnson,Cathleen,E.,1003 Justin Ln Apt 2016,NaN,Austin,TX,787572648,-14.76,2008-04-21,32
132,ABREU,KEVIN,M.,1305 GARDEN GLEN LANE,NaN,PEARLAND,TX,775816547,50.00,2007-09-30,35
133,ABREU,KEVIN,M.,1305 GARDEN GLEN LANE,NaN,PEARLAND,TX,775816547,150.00,2007-08-09,35
134,ABREU,KEVIN,M.,1305 GARDEN GLEN LANE,NaN,PEARLAND,TX,775816547,50.00,2007-07-19,35


,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
26,Buckler,Steve,NaN,24351 Armada Dr,NaN,Dana Point,CA,926291306,25.0,2007-08-16,20
49,Harrison,Ryan,NaN,2247 3rd St,NaN,La Verne,CA,917504918,25.0,2007-07-26,20
101,Aarons,Elaine,NaN,481 Buck Island Rd Apt 17A,APT 17A,West Yarmouth,MA,26733300,25.0,2008-02-26,34
140,ABEGG,PATRICIA,T.,1862 E. 5150 S.,NaN,SALT LAKE CITY,UT,841176911,25.0,2007-09-17,35
143,ABEGG,PATRICIA,T.,1862 E. 5150 S.,NaN,SALT LAKE CITY,UT,841176911,25.0,2007-08-06,35
144,ABEGG,PATRICIA,T.,1862 E. 5150 S.,NaN,SALT LAKE CITY,UT,841176911,25.0,2007-07-10,35
158,ABBO,PAULINE,MORENCY,10720 JACOB LANE,NaN,WHITE LAKE,MI,483862274,35.0,2008-01-07,37
160,ABAIR,PETER,NaN,40 EVANS STREET,NaN,WATERTOWN,MA,24722150,25.0,2008-01-09,37


In [30]:
# Sorting

dfcand.sort_values(by='party')
dfcand.sort_values(by='party', ascending=False) # Descending

# Column Select
dfcand[['last_name', 'party']]

# Counting
dfcand[['last_name', 'party']].count()

# Dropping Duplicates
dfcand[['first_name']].drop_duplicates()

# Count Dropped Duplicates
dfcand[['first_name']].drop_duplicates().count()

# Altering Tables
dfcand['name'] = dfcand['last_name'] + ", " + dfcand['first_name']
dfcand

# Changing Existing Fields
dfcand.loc[dfcand.first_name == "Mike", "name"]

6      Gravel, Mike
7    Huckabee, Mike
Name: name, dtype: object